In [2]:
from IPython.display import Video

- **MateGen项目演示**

In [3]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/4.MateGen%20Pro%20%E9%A1%B9%E7%9B%AE%E5%8A%9F%E8%83%BD%E6%BC%94%E7%A4%BA.mp4", width=800, height=400)

- **智能客服项目演示**

In [4]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/%E6%99%BA%E8%83%BD%E5%AE%A2%E6%9C%8D%E6%A1%88%E4%BE%8B%E6%BC%94%E7%A4%BA.mp4", width=800, height=400)

- **Dify项目演示**

In [6]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/2f1b47f42c65fd59e8d3a83e6cb9f13b_raw.mp4", width=800, height=400)

- **LangChain&LangGraph搭建Multi-Agnet**

In [7]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/%E5%8F%AF%E8%A7%86%E5%8C%96%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90Multi-Agent%E6%95%88%E6%9E%9C%E6%BC%94%E7%A4%BA%E6%95%88%E6%9E%9C.mp4", width=800, height=400)

---

# <center> LangChain快速入门与Agent开发实战
# <center> Part 4.LangChain记忆存储与搭建多轮对话机器人

In [1]:
import os
from dotenv import load_dotenv 
load_dotenv(override=True)

True

### 1. 构建多轮对话的流式智能问答系统

&emsp;&emsp;在`langChain`中构建一个基本的问答机器人仅需要使用一个`Chain`便可以快速实现，如下所示：

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model


chatbot_prompt = ChatPromptTemplate.from_messages([
    ("system", "你叫小智，是一名乐于助人的助手。"),
    ("user", "{input}")
])

# 使用 DeepSeek 模型
model = init_chat_model(model="deepseek-chat", model_provider="deepseek")  

# 直接使用模型 + 输出解析器
basic_qa_chain = chatbot_prompt | model | StrOutputParser()

# 测试
question = "你好，请你介绍一下你自己。"
result = basic_qa_chain.invoke(question)
print(result)

你好！我是小智，一个智能助手，随时准备为你提供帮助。我可以回答你的问题、提供信息、协助解决问题，或者陪你聊聊天。无论是日常生活、学习、工作，还是兴趣爱好，我都会尽力为你提供有用的建议和有趣的互动。有什么想问的，尽管告诉我吧！ 😊


- 添加多轮对话记忆

&emsp;&emsp;在LangChain中，我们可以通过人工拼接消息队列，来为每次模型调用设置多轮对话记忆。

In [3]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [4]:
chatbot_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="你叫小智，是一名乐于助人的助手。"
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [5]:
basic_qa_chain = chatbot_prompt | model | StrOutputParser()

In [6]:
messages_list = [
    HumanMessage(content="你好，我叫陈明，好久不见。"),
    AIMessage(content="你好呀！我是小智，一名乐于助人的AI助手。很高兴认识你！"),
]

In [7]:
question = "你好，请问我叫什么名字。"

In [8]:
messages_list.append(HumanMessage(content=question))

In [9]:
messages_list

[HumanMessage(content='你好，我叫陈明，好久不见。', additional_kwargs={}, response_metadata={}),
 AIMessage(content='你好呀！我是小智，一名乐于助人的AI助手。很高兴认识你！', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好，请问我叫什么名字。', additional_kwargs={}, response_metadata={})]

In [10]:
result = basic_qa_chain.invoke({"messages": messages_list})
print(result)

你刚刚告诉我你叫陈明呀！很高兴认识你，陈明！有什么我可以帮你的吗？ 😊


完整的多轮对话函如下：

In [11]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

model  = init_chat_model(model="deepseek-chat", model_provider="deepseek")
parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="你叫小智，是一名乐于助人的助手。"),
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt | model | parser

messages_list = []  # 初始化历史
print("🔹 输入 exit 结束对话")
while True:
    user_query = input("👤 你：")
    if user_query.lower() in {"exit", "quit"}:
        break

    # 1) 追加用户消息
    messages_list.append(HumanMessage(content=user_query))

    # 2) 调用模型
    assistant_reply = chain.invoke({"messages": messages_list})
    print("🤖 小智：", assistant_reply)

    # 3) 追加 AI 回复
    messages_list.append(AIMessage(content=assistant_reply))

    # 4) 仅保留最近 50 条
    messages_list = messages_list[-50:]


🔹 输入 exit 结束对话
🤖 小智： 你好，邹一苇！很高兴认识你。请问有什么可以帮你的吗？
🤖 小智： 25岁是充满可能性的年纪呢！刚开始工作可能会有些挑战，但也是快速学习和成长的黄金时期。如果需要交流职场适应、时间管理或是任何方面的经验，我很乐意为你提供建议～ 😊 目前从事什么行业呢？
🤖 小智： 很棒的选择！金融科技领域现在发展非常迅速，基金行业尤其需要技术人才来支持数据分析、交易系统和风控平台的开发。作为刚入行的开发者，或许可以多关注：

1. **金融业务知识** - 了解基金净值计算、投资组合管理等基础概念会对开发更有帮助
2. **技术栈特点** - 金融系统通常对并发处理、数据一致性有较高要求
3. **合规安全** - 金融行业对数据安全和监管合规非常重视

最近在接触什么类型的项目呢？遇到具体技术问题时也欢迎随时交流~ 💻
🤖 小智： 当然记得！您刚才提到过：  
**姓名**：邹一苇  
**年龄**：25岁  
**职业**：软件开发工程师（目前就职于基金行业）  

如果有需要补充或修正的地方，请随时告诉我～ 😊


- 流式打印聊天信息

&emsp;&emsp;此外还有一个问题是，大家经常看到的问答机器人其实都是采用流式传输模式。用户输入问题，等待模型直接返回回答，然后用户再输入问题，模型再返回回答，这样循环下去，用户输入问题和模型返回回答之间的时间间隔太长，导致用户感觉机器人反应很慢。所以`LangChain`提供了一个`astream`方法，可以实现流式输出，即一旦模型有输出，就立即返回，这样用户就可以看到模型正在思考，而不是等待模型思考完再返回。


&emsp;&emsp;实现的方法也非常简单，只需要在调用模型时将`invoke`方法替换为`astream`方法，然后使用`async for`循环来获取模型的输出即可。代码如下：

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate


chatbot_prompt = ChatPromptTemplate.from_messages([
    ("system", "你叫小智，是一名乐于助人的助手。"),
    ("user", "{input}")
])

# 使用 DeepSeek 模型
model = init_chat_model(model="deepseek-chat", model_provider="deepseek")  

# 直接使用提示模版 +模型 + 输出解析器
qa_chain_with_system = chatbot_prompt | model | StrOutputParser()

# 异步实现流式输出
async for chunk in qa_chain_with_system.astream({"input": "你好，请你介绍一下你自己"}):
    print(chunk, end="", flush=True)

你好！我是小智，一名智能助手，随时准备为你提供帮助。我可以回答你的问题、提供信息、协助解决问题，或者陪你聊天。无论是学习、工作还是生活中的疑问，我都会尽力给出清晰、准确的回答。我的知识覆盖广泛，包括科技、文化、教育、娱乐等多个领域，并且会不断更新。如果有任何需要，请随时告诉我！ 😊

In [38]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="你叫小智，是一名乐于助人的助手。"),
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt | model | parser

messages_list = []  # 初始化历史
print("🔹 输入 exit 结束对话")
while True:
    user_query = input("👤 你：")
    if user_query.lower() in {"exit", "quit"}:
        break

    # 1) 追加用户消息
    messages_list.append(HumanMessage(content=user_query))

    # 2) 调用模型
    async for chunk in chain.astream({"messages": messages_list}):
        print(chunk, end="", flush=True)

    # 3) 追加 AI 回复
    messages_list.append(AIMessage(content=assistant_reply))

    # 4) 仅保留最近 50 条
    messages_list = messages_list[-50:]

🔹 输入 exit 结束对话


👤 你： 你好，我叫陈明，好久不见


你好啊陈明！确实好久不见了，最近过得怎么样？工作还顺利吗？记得上次聊天时你好像正在准备一个重要的项目，现在应该已经顺利完成了吧？有什么新鲜事想和我分享的吗？

👤 你： 请问，你还记得我叫什么名字么？


（突然进入「名侦探模式」）  

陈明同学，这可是道送分题！✨ 虽然我的记忆像金鱼一样只有7秒，但当前对话中你刚刚强调过——  

**「陈明」** 这两个字已经用荧光笔标在我脑海的小黑板上了！(๑•̀ㅂ•́)و✧  

（不过如果现在你突然说「其实我叫张大勇」…我也会立刻乖巧改口的hhh）

👤 你： exit


&emsp;&emsp;如上所示展示的问答效果就是我们在构建大模型应用时需要实现的流式输出效果。接下来我们就进一步地，使用`gradio`来开发一个支持在网页上进行交互的问答机器人。

&emsp;&emsp;首先需要安装一下`gradio`的第三方依赖包，

In [31]:
# 安装 Gradio
! pip install gradio

  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.6.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.10.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached pillow-11.2.1-cp312-cp312-win_amd64.whl.metadata (9.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached typer-0.16.0-py3-none-any.whl.metadata (15 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached websockets-15.0.1-cp312-cp312-win_

&emsp;&emsp;完整实现的代码如下：

In [17]:
import gradio as gr
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# ──────────────────────────────────────────────
# 1. 模型、Prompt、Chain
# ──────────────────────────────────────────────
model = init_chat_model("deepseek-chat", model_provider="deepseek")
parser = StrOutputParser()

chatbot_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="你叫小智，是一名乐于助人的助手。"),
        MessagesPlaceholder(variable_name="messages"),  # 手动传入历史
    ]
)

qa_chain = chatbot_prompt | model | parser   # LCEL 组合

# ──────────────────────────────────────────────
# 2. Gradio 组件
# ──────────────────────────────────────────────
CSS = """
.main-container {max-width: 1200px; margin: 0 auto; padding: 20px;}
.header-text {text-align: center; margin-bottom: 20px;}
"""

def create_chatbot() -> gr.Blocks:
    with gr.Blocks(title="DeepSeek Chat", css=CSS) as demo:
        with gr.Column(elem_classes=["main-container"]):
            gr.Markdown("# 🤖 LangChain B站公开课 By九天Hector", elem_classes=["header-text"])
            gr.Markdown("基于 LangChain LCEL 构建的流式对话机器人", elem_classes=["header-text"])

            chatbot = gr.Chatbot(
                height=500,
                show_copy_button=True,
                avatar_images=(
                    "https://cdn.jsdelivr.net/gh/twitter/twemoji@v14.0.2/assets/72x72/1f464.png",
                    "https://cdn.jsdelivr.net/gh/twitter/twemoji@v14.0.2/assets/72x72/1f916.png",
                ),
            )
            msg = gr.Textbox(placeholder="请输入您的问题...", container=False, scale=7)
            submit = gr.Button("发送", scale=1, variant="primary")
            clear = gr.Button("清空", scale=1)

        # ---------------  状态：保存 messages_list  ---------------
        state = gr.State([])          # 这里存放真正的 Message 对象列表

        # ---------------  主响应函数（流式） ----------------------
        async def respond(user_msg: str, chat_hist: list, messages_list: list):
            # 1) 输入为空直接返回
            if not user_msg.strip():
                yield "", chat_hist, messages_list
                return

            # 2) 追加用户消息
            messages_list.append(HumanMessage(content=user_msg))
            chat_hist = chat_hist + [(user_msg, None)]
            yield "", chat_hist, messages_list      # 先显示用户消息

            # 3) 流式调用模型
            partial = ""
            async for chunk in qa_chain.astream({"messages": messages_list}):
                partial += chunk
                # 更新最后一条 AI 回复
                chat_hist[-1] = (user_msg, partial)
                yield "", chat_hist, messages_list

            # 4) 完整回复加入历史，裁剪到最近 50 条
            messages_list.append(AIMessage(content=partial))
            messages_list = messages_list[-50:]

            # 5) 最终返回（Gradio 需要把新的 state 传回）
            yield "", chat_hist, messages_list

        # ---------------  清空函数 -------------------------------
        def clear_history():
            return [], "", []          # 清空 Chatbot、输入框、messages_list

        # ---------------  事件绑定 ------------------------------
        msg.submit(respond, [msg, chatbot, state], [msg, chatbot, state])
        submit.click(respond, [msg, chatbot, state], [msg, chatbot, state])
        clear.click(clear_history, outputs=[chatbot, msg, state])

    return demo


# ──────────────────────────────────────────────
# 3. 启动应用
# ──────────────────────────────────────────────
demo = create_chatbot()
demo.launch(server_name="0.0.0.0", server_port=None, share=False, debug=True)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_26808\1965123637.py:36: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7860): [winerror 10048] 通常每个套接字地址(协议/网络地址/端口)只允许使用一次。
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7861): [winerror 10048] 通常每个套接字地址(协议/网络地址/端口)只允许使用一次。


* Running on local URL:  http://0.0.0.0:7862


Exception: Couldn't start the app because 'http://localhost:7862/gradio_api/startup-events' failed (code 502). Check your network or proxy settings to ensure localhost is accessible.

&emsp;&emsp;运行后，在浏览器访问`http://127.0.0.1:7860`即可进行问答交互。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506121740968.png" alt="image-20250612174010864" style="zoom:50%;" />

具体代码解释如下：

##### 🧱 1. 模块说明

```python
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
import gradio as gr
```

* `init_chat_model`：初始化 DeepSeek 等聊天模型。
* `ChatPromptTemplate`：用于构建聊天 Prompt 模板。
* `MessagesPlaceholder`：用于占位历史消息。
* `HumanMessage` / `AIMessage`：构建多轮消息结构。
* `StrOutputParser`：将模型输出转换为字符串。
* `gradio`：构建网页界面。

---

##### 🧠 2. Prompt 构建与模型初始化

```python
model = init_chat_model("deepseek-chat", model_provider="deepseek")
parser = StrOutputParser()

chatbot_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="你叫小智，是一名乐于助人的助手。"),
    MessagesPlaceholder(variable_name="messages"),
])

qa_chain = chatbot_prompt | model | parser
```

* **SystemMessage**：初始化系统角色设定（小智）。
* **MessagesPlaceholder**：用变量名 `messages` 占位历史消息。
* **qa\_chain**：组合为 LangChain Expression Language 链。

---

##### 🔄 3. 手动管理消息列表

```python
state = gr.State([])
```

我们用 `gr.State` 存储所有历史消息（列表）。每次用户发送消息，都会：

* append 一个 `HumanMessage`。
* 流式调用模型并不断更新回复。
* append 一个 `AIMessage`。
* 最后裁剪：`messages_list = messages_list[-50:]`。

---

##### 🌊 4. 流式响应函数

```python
async def respond(user_msg: str, chat_hist: list, messages_list: list):
    if not user_msg.strip():
        yield "", chat_hist, messages_list
        return

    messages_list.append(HumanMessage(content=user_msg))
    chat_hist = chat_hist + [(user_msg, None)]
    yield "", chat_hist, messages_list

    partial = ""
    async for chunk in qa_chain.astream({"messages": messages_list}):
        partial += chunk
        chat_hist[-1] = (user_msg, partial)
        yield "", chat_hist, messages_list

    messages_list.append(AIMessage(content=partial))
    messages_list = messages_list[-50:]
    yield "", chat_hist, messages_list
```

* **支持 async 流式输出**。
* **动态更新最后一轮对话**。
* **通过 `yield` 实时反馈到前端**。

---

##### 🧼 5. 清空历史函数

```python
def clear_history():
    return [], "", []
```

用于点击 "清空" 按钮时重置历史记录、输入框和消息状态。

---

##### 🧩 6. Gradio 界面构建

```python
msg.submit(respond, [msg, chatbot, state], [msg, chatbot, state])
submit.click(respond, [msg, chatbot, state], [msg, chatbot, state])
clear.click(clear_history, outputs=[chatbot, msg, state])
```

* **事件绑定**：用户提交文本 → 调用 `respond` → 返回新状态。
* **Gradio Chatbot 组件**：使用 `avatar_images` 设置人机头像。
* **Gradio State**：跨组件共享并持久化消息列表。

---

##### ✅ 总结

| 功能模块      | 实现方式                                              |
| --------- | ------------------------------------------------- |
| 对话模型      | DeepSeek via `init_chat_model`                    |
| Prompt 模板 | ChatPromptTemplate + System + MessagesPlaceholder |
| 消息管理      | 手动管理 + `gr.State` 保存并裁剪最近 50 条                    |
| 多轮对话      | 用户/AI Message 列表构建并传入 LCEL 链                      |
| UI 界面     | Gradio Blocks + Chatbot 组件 + 清空按钮                 |
| 流式输出      | 使用 `qa_chain.astream()` 持续生成回复                    |

当然这只是最简单的问答机器人实现形式，实际上企业应用的问答机器人往往需要更加复杂的逻辑，比如用户权限管理、上下文记忆等，更多内容详见《大模型与Agent开发》课程讲解。